In [1]:
from stable_baselines3 import A2C
from stable_baselines3.common.env_util import make_atari_env
from stable_baselines3.common.vec_env import VecFrameStack

2023-06-12 16:43:15.303834: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-06-12 16:43:15.350017: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-12 16:43:16.106226: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
env_id = "Breakout-v4"
gif_folder = "logfiles/gifs/"
models_folder = "models"

# There already exists an environment generator that will make and wrap atari environments correctly.
env = make_atari_env(env_id, n_envs=4, seed=0)
# Stack 4 frames
env = VecFrameStack(env, n_stack=4)

A.L.E: Arcade Learning Environment (version 0.8.1+53f58b7)
[Powered by Stella]


In [3]:
model = A2C("MlpPolicy", env, verbose=1)
model.learn(log_interval=1E5, total_timesteps=2E7, progress_bar=True)



Using cuda device
Wrapping the env in a VecTransposeImage.


Output()

-------------------------------------
| rollout/              |           |
|    ep_len_mean        | 313       |
|    ep_rew_mean        | 2.11      |
| time/                 |           |
|    fps                | 197       |
|    iterations         | 100000    |
|    time_elapsed       | 10126     |
|    total_timesteps    | 2000000   |
| train/                |           |
|    entropy_loss       | -1.54e-05 |
|    explained_variance | -3.58e-07 |
|    learning_rate      | 0.0007    |
|    n_updates          | 99999     |
|    policy_loss        | -7.08e-08 |
|    value_loss         | 0.0226    |
-------------------------------------
-------------------------------------
| rollout/              |           |
|    ep_len_mean        | 294       |
|    ep_rew_mean        | 1.77      |
| time/                 |           |
|    fps                | 200       |
|    iterations         | 200000    |
|    time_elapsed       | 19904     |
|    total_timesteps    | 4000000   |
| train/    

In [4]:
model.save(f"{models_folder}/a2c_vec_{env_id}")

In [5]:
env = make_atari_env(env_id, n_envs=1, seed=0)
env = VecFrameStack(env, n_stack=4)

In [6]:
images = []
obs = env.reset()
img = env.render(mode="rgb_array")
for i in range(500):
# done = False
# while not done:
    images.append(img)
    action, _ = model.predict(obs)
    obs, reward, done, info = env.step(action)
    img = env.render(mode="rgb_array")

/usr/local/lib/python3.10/dist-packages/gymnasium/utils/passive_env_checker.py:364: UserWarning: WARN: No render fps was declared in the environment (env.metadata['render_fps'] is None or not defined), rendering may occur at inconsistent fps.
  logger.warn(


In [7]:
import imageio
import numpy as np

In [8]:
# fps = 60*4
# dur = len(images)/fps
dur = 50
imageio.mimsave(f"{gif_folder}/{env_id}_vec.gif", [np.array(img) for i, img in enumerate(images) if i%2 == 0], duration=dur)